# Datathon — Engenharia de Features

Pipeline de construção de variáveis para o modelo de alerta preventivo de risco educacional. Transforma os pares longitudinais gerados pelo `preprocessing.ipynb` em datasets prontos para treino e validação.


## Visão Geral

| Etapa | Descrição |
|---|---|
| **1. Carregamento** | Lê `train_df.parquet` e `valid_df.parquet` gerados no `preprocessing.ipynb` |
| **2. Engenharia de Features** | `make_features` constrói variáveis individuais, de grupo e derivadas |
| **3. Inspeção de Qualidade** | Verifica distribuição do target, taxa de nulos e colunas resultantes |
| **4. Limpeza Pré-Modelo** | Remove colunas 100% nulas e identificadores sem valor preditivo |
| **5. Exportação** | Persiste `train_feat` e `valid_feat` em Parquet + metadados em JSON |

---

## Decisões de Design

| Decisão | Justificativa |
|---|---|
| **Lookup calculado apenas no treino** | Evita data leakage — estatísticas de grupo não podem ser recalculadas com dados de validação ou inferência |
| **Inglês neutralizado em fases sem obrigatoriedade** (`ALFA`, `FASE1`, `FASE2`, `FASE8`) | Zera o campo onde inglês não é avaliado, eliminando viés sistemático por ausência |
| **`defasagem_t` como feature, não como filtro** | Inclui todos os alunos elegíveis no treino — o estado inicial é informação, não critério de exclusão |
| **Fallback turma → fase → global** | Garante que alunos em turmas pequenas ou novas recebam estimativas razoáveis em vez de `NaN` |
| **Exportação sem `ra`, `pair_label`, `fase_raw`** | Colunas de identificação e metadados não têm poder preditivo e não devem entrar no modelo |

---

## Saídas deste Notebook

| Arquivo | Conteúdo |
|---|---|
| `train_feat__piora__2022_2023.parquet` | Dataset de treino com todas as features e target |
| `valid_feat__piora__2023_2024.parquet` | Dataset de validação com todas as features e target |
| `meta__piora__2022_2023__2023_2024.json` | Metadados do experimento (dimensões, definição do target, colunas removidas) |


## 1. Carregamento dos Datasets

Lê os parquets gerados pelo `preprocessing.ipynb` — pares longitudinais T→T+1 com features do ano T e target derivado do ano T+1.

| Arquivo | Conteúdo |
|---|---|
| `train_df.parquet` | Pares 2022→2023 com `defasagem_t`, `defasagem_t1` e `target` |
| `valid_df.parquet` | Par 2023→2024 com `defasagem_t`, `defasagem_t1` e `target` |


In [58]:
import pandas as pd
from pathlib import Path

# Caminhos locais
ROOT_DIR = Path("..").resolve()          # notebooks/ -> workspace root
NOTEBOOKS_DIR = ROOT_DIR / "notebooks"  # onde os parquets foram salvos

train_df = pd.read_parquet(NOTEBOOKS_DIR / "data" / "train_df.parquet")
valid_df = pd.read_parquet(NOTEBOOKS_DIR / "data" / "valid_df.parquet")

print("✅ Dados carregados de:", NOTEBOOKS_DIR / "data")
print("  train_df:", train_df.shape, "  valid_df:", valid_df.shape)


✅ Dados carregados de: /home/glauberthy/Desktop/datathon/notebooks/data
  train_df: (600, 22)   valid_df: (765, 22)


In [59]:
valid_df.shape, train_df.shape

((765, 22), (600, 22))

In [60]:
valid_df['fase'].unique()

<IntegerArray>
[8, 0, 1, 5, 2, 3, 4, 7, 6]
Length: 9, dtype: Int64

## 2. Pipeline de Engenharia de Features

Toda a lógica está centralizada na função `make_features`, que recebe um DataFrame e um dicionário `lookup` (estatísticas de grupo calculadas no treino) e retorna o dataset enriquecido.

### Etapas internas de `make_features`

| Etapa | Descrição |
|---|---|
| **1. Base acadêmica** | Média e dispersão das provas (`matem`, `portug`, `ingles`); flag `fez_ingles` |
| **2. Regra do inglês** | Zera `ingles` nas fases sem obrigatoriedade (`ALFA`, `FASE1`, `FASE2`, `FASE8`) |
| **3. Contexto** | `tempo_casa` = `ano_base` − `ano_ingresso` |
| **4. Estado atual** | `defasagem_t` fatiado em 3 bins: igual a `0`, igual a `1`, maior ou igual a `2` |
| **5. Lookup (treino only)** | Calcula média, desvio-padrão, p25 e p75 por turma, fase e global — **somente no treino** |
| **6. Deltas e z-scores** | Desempenho relativo do aluno em relação à turma e à fase (`delta_*`, `z_*`) |
| **7. Flags de risco** | `abaixo_p25_turma_*` — aluno abaixo do quartil inferior da turma |
| **8. Seleção final** | Mantém IDs, features base e derivadas; exclui `defasagem_t1` explicitamente (anti-leakage) |

> O `lookup` calculado no treino é reutilizado na validação sem qualquer recalibração com dados futuros.


In [61]:
import numpy as np
import pandas as pd

EPS = 1e-6
PHASES_ENGLISH_NOT_REQUIRED_INT = {0, 1, 2, 8}  # ajuste se seu "ALFA" não for 0

def _p25(x):
    return np.nanquantile(x, 0.25)

def _p75(x):
    return np.nanquantile(x, 0.75)

def make_features(df: pd.DataFrame, lookup: dict | None, is_train: bool):
    df = df.copy()

    # -------------------------
    # 1) Base acadêmica
    # -------------------------
    df["fez_ingles"] = df["ingles"].notna().astype(int)

    provas_cols = ["matem", "portug", "ingles"]
    df["media_provas"] = df[provas_cols].mean(axis=1, skipna=True)
    df["disp_provas"]  = df[provas_cols].std(axis=1, ddof=0, skipna=True)

    # -------------------------
    # 2) Regra do inglês por fase (neutraliza onde não obrigatório)
    # -------------------------
    mask_no_eng = df["fase"].isin(PHASES_ENGLISH_NOT_REQUIRED_INT)
    df.loc[mask_no_eng, "ingles"] = np.nan
    df.loc[mask_no_eng, "fez_ingles"] = 0

    # -------------------------
    # 3) Contexto
    # -------------------------
    df["tempo_casa"] = df["ano_base"] - df["ano_ingresso"]

    # -------------------------
    # 4) Estratégia nova: estado atual (defasagem_t) como features
    # -------------------------
    d = df["defasagem_t"].fillna(0)
    df["defasagem_t_is0"] = (d == 0).astype(int)
    df["defasagem_t_bin_1"] = (d == 1).astype(int)
    df["defasagem_t_bin_2plus"] = (d >= 2).astype(int)

    # -------------------------
    # 5) Lookups (turma / fase / global) — TREINO ONLY
    # -------------------------
    group_vars = ["media_provas", "matem", "portug", "ieg", "ips", "iaa"]

    if is_train:
        turma_stats = (
            df.groupby(["ano_base", "turma"], dropna=False)[group_vars]
              .agg(["mean", "std", _p25, _p75])
        )
        turma_stats.columns = [f"turma_{stat}_{col}" for col, stat in turma_stats.columns]
        turma_stats = turma_stats.reset_index()

        fase_stats = (
            df.groupby(["ano_base", "fase"], dropna=False)[group_vars]
              .agg(["mean", "std", _p25, _p75])
        )
        fase_stats.columns = [f"fase_{stat}_{col}" for col, stat in fase_stats.columns]
        fase_stats = fase_stats.reset_index()

        global_stats = (
            df.groupby(["ano_base"], dropna=False)[group_vars]
              .agg(["mean", "std", _p25, _p75])
        )
        global_stats.columns = [f"global_{stat}_{col}" for col, stat in global_stats.columns]
        global_stats = global_stats.reset_index()

        lookup = {"turma": turma_stats, "fase": fase_stats, "global": global_stats}

    # aplica lookup (sem recalcular em validação)
    df = df.merge(lookup["turma"],  on=["ano_base", "turma"], how="left")
    df = df.merge(lookup["fase"],   on=["ano_base", "fase"],  how="left")
    df = df.merge(lookup["global"], on=["ano_base"],          how="left")

    # fallback: turma -> fase -> global (para cada stat)
    for col in group_vars:
        for stat in ["mean", "std", "_p25", "_p75"]:
            c_turma  = f"turma_{stat}_{col}"
            c_fase   = f"fase_{stat}_{col}"
            c_global = f"global_{stat}_{col}"
            df[c_turma] = df[c_turma].fillna(df[c_fase]).fillna(df[c_global])

    # -------------------------
    # 6) deltas e z-scores (turma e fase)
    # -------------------------
    for col in group_vars:
        # turma
        m = f"turma_mean_{col}"
        s = f"turma_std_{col}"
        df[f"delta_turma_{col}"] = df[col] - df[m]
        denom = df[s].where(df[s] > EPS)  # std pequeno vira NaN
        df[f"z_turma_{col}"] = ((df[col] - df[m]) / denom).fillna(0.0)

        # fase
        m = f"fase_mean_{col}"
        s = f"fase_std_{col}"
        df[f"delta_fase_{col}"] = df[col] - df[m]
        denom = df[s].where(df[s] > EPS)
        df[f"z_fase_{col}"] = ((df[col] - df[m]) / denom).fillna(0.0)

    # -------------------------
    # 7) flags p25 (turma) para algumas variáveis
    # -------------------------
    for col in ["media_provas", "matem", "ieg"]:
        p25 = f"turma__p25_{col}"   # porque stat="_p25"
        if p25 in df.columns:
            df[f"abaixo_p25_turma_{col}"] = (df[col] < df[p25]).astype("Int64")

    # -------------------------
    # 8) Seleção final de features (mantém alvo e ids para rastreio)
    # -------------------------
    id_cols = ["ra", "ano_base", "pair_label", "fase", "turma", "target"]
    base_cols = [
        "genero", "instituicao", "escola", "ano_ingresso", "tempo_casa",
        "ano_nasc", "idade",
        "defasagem_t", "defasagem_t_is0", "defasagem_t_bin_1", "defasagem_t_bin_2plus",
        "matem", "portug", "ingles", "fez_ingles", "media_provas", "disp_provas",
        "ieg", "iaa", "ips", "ipp"
    ]
    derived_cols = [c for c in df.columns if c.startswith(("turma_", "fase_", "global_", "delta_", "z_", "abaixo_p25_"))]

    keep = [c for c in id_cols if c in df.columns] + [c for c in base_cols if c in df.columns] + derived_cols

    # Segurança: nunca deixe futuro entrar como feature
    keep = [c for c in keep if c != "defasagem_t1"]

    return df[keep].copy(), lookup

In [62]:
train_feat, lookup = make_features(train_df, lookup=None, is_train=True)
valid_feat, _      = make_features(valid_df, lookup=lookup, is_train=False)

train_feat.shape, valid_feat.shape

((600, 127), (765, 127))

## 3. Inspeção de Qualidade

Verificações antes de finalizar o dataset:

| Checagem | O que observar |
|---|---|
| **Base rate do target** | Treino e validação devem ter taxa de piora não nula e não trivial (entre 10 % e 90 %) |
| **Taxa de nulos por feature** | Features com alta proporção de nulos podem prejudicar o modelo ou indicar erro no pipeline |
| **Colunas 100 % nulas** | Devem ser removidas — provavelmente variáveis ausentes nos anos do par selecionado |
| **Schema treino = validação** | As colunas devem ser idênticas antes da entrega ao modelo |


In [63]:
train_feat["target"].mean(), valid_feat["target"].mean()

(np.float64(0.30833333333333335), np.float64(0.4091503267973856))

In [64]:
train_feat.isna().mean().sort_values(ascending=False).head(15)

escola          1.000000
ipp             1.000000
ingles          0.698333
fase            0.000000
turma           0.000000
ano_base        0.000000
ra              0.000000
genero          0.000000
instituicao     0.000000
ano_ingresso    0.000000
tempo_casa      0.000000
ano_nasc        0.000000
idade           0.000000
target          0.000000
pair_label      0.000000
dtype: float64

In [65]:
train_feat['ipp'].isna().sum()

np.int64(600)

In [66]:
valid_feat['ipp'].isna().sum()

np.int64(72)

In [67]:
list(train_feat.columns)

['ra',
 'ano_base',
 'pair_label',
 'fase',
 'turma',
 'target',
 'genero',
 'instituicao',
 'escola',
 'ano_ingresso',
 'tempo_casa',
 'ano_nasc',
 'idade',
 'defasagem_t',
 'defasagem_t_is0',
 'defasagem_t_bin_1',
 'defasagem_t_bin_2plus',
 'matem',
 'portug',
 'ingles',
 'fez_ingles',
 'media_provas',
 'disp_provas',
 'ieg',
 'iaa',
 'ips',
 'ipp',
 'fase_raw',
 'turma_mean_media_provas',
 'turma_std_media_provas',
 'turma__p25_media_provas',
 'turma__p75_media_provas',
 'turma_mean_matem',
 'turma_std_matem',
 'turma__p25_matem',
 'turma__p75_matem',
 'turma_mean_portug',
 'turma_std_portug',
 'turma__p25_portug',
 'turma__p75_portug',
 'turma_mean_ieg',
 'turma_std_ieg',
 'turma__p25_ieg',
 'turma__p75_ieg',
 'turma_mean_ips',
 'turma_std_ips',
 'turma__p25_ips',
 'turma__p75_ips',
 'turma_mean_iaa',
 'turma_std_iaa',
 'turma__p25_iaa',
 'turma__p75_iaa',
 'fase_mean_media_provas',
 'fase_std_media_provas',
 'fase__p25_media_provas',
 'fase__p75_media_provas',
 'fase_mean_matem',


In [68]:
train_feat.shape, valid_feat.shape

((600, 127), (765, 127))

In [69]:
# remove colunas 100% NaN automaticamente
all_nan_cols = train_feat.columns[train_feat.isna().all()].tolist()
print("Colunas 100% NaN:", all_nan_cols)

train_feat = train_feat.drop(columns=all_nan_cols)
valid_feat = valid_feat.drop(columns=all_nan_cols)

Colunas 100% NaN: ['escola', 'ipp']


In [70]:
train_feat.shape, valid_feat.shape


((600, 125), (765, 125))

In [71]:
train_feat.isna().mean().sort_values(ascending=False).head(10)

ingles          0.698333
ano_base        0.000000
pair_label      0.000000
fase            0.000000
ra              0.000000
target          0.000000
genero          0.000000
instituicao     0.000000
ano_ingresso    0.000000
tempo_casa      0.000000
dtype: float64

## 4. Limpeza Pré-Modelo

Remove colunas que não devem entrar no modelo por serem identificadores ou metadados sem poder preditivo:

| Coluna | Motivo da remoção |
|---|---|
| `ra` | Identificador do aluno — sem generalização preditiva |
| `pair_label` | Metadado do par longitudinal (ex.: `"2022->2023"`) |
| `fase_raw` | Versão textual bruta de `fase` — substituída pela coluna numérica normalizada |

> Colunas 100 % nulas detectadas na seção anterior já foram removidas antes desta etapa.


In [72]:
COLS_DROP_PRE_MODEL = ["ra", "pair_label", "fase_raw"]

train_feat = train_feat.drop(columns=[c for c in COLS_DROP_PRE_MODEL if c in train_feat.columns])
valid_feat = valid_feat.drop(columns=[c for c in COLS_DROP_PRE_MODEL if c in valid_feat.columns])

In [73]:
assert "ra" not in train_feat.columns
assert "pair_label" not in train_feat.columns
assert "fase_raw" not in train_feat.columns

assert set(train_feat.columns) == set(valid_feat.columns)

## 5. Exportação

Persiste os datasets finais e registra os metadados do experimento para rastreabilidade:

| Arquivo | Conteúdo |
|---|---|
| `train_feat__piora__2022_2023.parquet` | Dataset de treino: pares 2022→2023 com features e target |
| `valid_feat__piora__2023_2024.parquet` | Dataset de validação: par 2023→2024 com features e target |
| `meta__piora__2022_2023__2023_2024.json` | Metadados: dimensões, definição do target, colunas removidas, timestamp |

> Os arquivos Parquet são a entrada direta do `train.ipynb`. O JSON de metadados documenta o experimento para reprodutibilidade.


In [74]:
DATA_DIR = NOTEBOOKS_DIR / "data"
MODELS_DIR = NOTEBOOKS_DIR / "models"
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

train_feat.to_parquet(
    DATA_DIR / "train_feat__piora__2022_2023.parquet",
    index=False
)

valid_feat.to_parquet(
    DATA_DIR / "valid_feat__piora__2023_2024.parquet",
    index=False
)

print("✅ Features salvas em:", DATA_DIR)
print(" - train_feat__piora__2022_2023.parquet")
print(" - valid_feat__piora__2023_2024.parquet")


✅ Features salvas em: /home/glauberthy/Desktop/datathon/notebooks/data
 - train_feat__piora__2022_2023.parquet
 - valid_feat__piora__2023_2024.parquet


In [75]:
import json
from datetime import datetime

meta = {
    "created_at": datetime.now().isoformat(),
    "storage": str(DATA_DIR),
    "train_pairs": ["2022->2023"],
    "valid_pairs": ["2023->2024"],
    "target_definition": "target = 1 if defasagem_t1 > defasagem_t else 0",
    "dropped_columns": ["ra", "pair_label", "fase_raw", "escola", "ipp"],
    "n_train": int(len(train_feat)),
    "n_valid": int(len(valid_feat)),
    "n_features": int(train_feat.drop(columns=["target"]).shape[1]),
}

with open(
    MODELS_DIR / "meta__piora__2022_2023__2023_2024.json",
    "w",
    encoding="utf-8"
) as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("✅ Meta salvo em:", MODELS_DIR / "meta__piora__2022_2023__2023_2024.json")


✅ Meta salvo em: /home/glauberthy/Desktop/datathon/notebooks/models/meta__piora__2022_2023__2023_2024.json
